# R-CNN with VOC 2007

This notebook demonstrates a simplified R-CNN pipeline using VGG16 for feature extraction and Linear SVM for classification, trained on real object detection data from the Pascal VOC dataset.

In [ ]:
!pip install opencv-python-headless tensorflow scikit-learn matplotlib tensorflow-datasets

In [ ]:
import cv2
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from sklearn.svm import LinearSVC
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt

In [1]:
from google.colab import files

# Upload VOCtrainval and VOCtest files
uploaded = files.upload()


KeyboardInterrupt: 

In [ ]:
import tarfile

# Extract the TAR files into the appropriate directory
with tarfile.open('VOCtrainval_06-Nov-2007.tar', 'r') as tar:
    tar.extractall()

with tarfile.open('VOCtest_06-Nov-2007.tar', 'r') as tar:
    tar.extractall()


In [ ]:
#Load Pascal VOC Dataset

# Define the local directory where the VOC dataset is stored (after extraction)
data_dir = '/content/VOC2007'  # Adjust this to your path

# Load the Pascal VOC 2007 dataset from the local directory
ds, ds_info = tfds.load(
    'voc/2007',
    data_dir=data_dir,  # Specify the local dataset directory
    split='train[:10%]',
    with_info=True,
    shuffle_files=True,
    download=False  # Disable automatic download
)

# Display dataset info
print(ds_info)


Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

TooManyRedirects: Exceeded 30 redirects.

In [ ]:
def preprocess_voc_sample(example):
    image = tf.image.resize(example['image'], (256, 256)).numpy()
    gt_boxes = example['objects']['bbox'].numpy()
    gt_labels = [obj['label'].numpy() for obj in example['objects']]
    h, w = image.shape[:2]
    abs_boxes = []
    for box in gt_boxes:
        ymin, xmin, ymax, xmax = box
        abs_boxes.append([
            int(xmin * w), int(ymin * h), int(xmax * w), int(ymax * h)
        ])
    return image.astype(np.uint8), abs_boxes, gt_labels

In [ ]:
def generate_sliding_windows(image, step=64, window_size=(128, 128)):
    windows = []
    h, w, _ = image.shape
    for y in range(0, h - window_size[1], step):
        for x in range(0, w - window_size[0], step):
            windows.append((x, y, x + window_size[0], y + window_size[1]))
    return windows

def compute_iou(boxA, boxB):
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])
    interArea = max(0, xB - xA) * max(0, yB - yA)
    boxAArea = (boxA[2] - boxA[0]) * (boxA[3] - boxA[1])
    boxBArea = (boxB[2] - boxB[0]) * (boxB[3] - boxB[1])
    iou = interArea / float(boxAArea + boxBArea - interArea)
    return iou


In [ ]:
vgg = VGG16(weights="imagenet", include_top=False, input_shape=(224, 224, 3))

def prepare_training_data(image, gt_boxes, iou_thresh=0.5):
    windows = generate_sliding_windows(image)
    X, y = [], []
    for window in windows:
        ious = [compute_iou(window, gt) for gt in gt_boxes]
        max_iou = max(ious) if ious else 0
        label = 1 if max_iou >= iou_thresh else 0
        patch = image[window[1]:window[3], window[0]:window[2]]
        if patch.shape[0] == 0 or patch.shape[1] == 0:
            continue
        resized = cv2.resize(patch, (224, 224))
        input_tensor = preprocess_input(np.expand_dims(resized, axis=0))
        feature = vgg.predict(input_tensor)
        X.append(feature.flatten())
        y.append(label)
    return np.array(X), np.array(y)

In [ ]:
X_train, y_train = [], []
for example in tfds.as_numpy(ds):
    img, gt_boxes, gt_labels = preprocess_voc_sample(example)
    x, y = prepare_training_data(img, gt_boxes)
    X_train.extend(x)
    y_train.extend(y)

clf = LinearSVC()
clf.fit(X_train, y_train)

In [ ]:
def non_max_suppression(boxes, scores, threshold=0.3):
    boxes = np.array(boxes)
    scores = np.array(scores)
    x1 = boxes[:, 0]; y1 = boxes[:, 1]; x2 = boxes[:, 2]; y2 = boxes[:, 3]
    areas = (x2 - x1) * (y2 - y1)
    order = scores.argsort()[::-1]
    keep = []
    while order.size > 0:
        i = order[0]
        keep.append(i)
        xx1 = np.maximum(x1[i], x1[order[1:]])
        yy1 = np.maximum(y1[i], y1[order[1:]])
        xx2 = np.minimum(x2[i], x2[order[1:]])
        yy2 = np.minimum(y2[i], y2[order[1:]])
        w = np.maximum(0.0, xx2 - xx1)
        h = np.maximum(0.0, yy2 - yy1)
        inter = w * h
        iou = inter / (areas[i] + areas[order[1:]] - inter)
        order = order[1:][iou <= threshold]
    return keep

In [ ]:
test_image, gt_boxes, _ = preprocess_voc_sample(next(iter(tfds.as_numpy(ds))))
windows = generate_sliding_windows(test_image)
features = []
boxes = []
for window in windows:
    patch = test_image[window[1]:window[3], window[0]:window[2]]
    if patch.shape[0] == 0 or patch.shape[1] == 0:
        continue
    resized = cv2.resize(patch, (224, 224))
    input_tensor = preprocess_input(np.expand_dims(resized, axis=0))
    feature = vgg.predict(input_tensor)
    features.append(feature.flatten())
    boxes.append(window)

scores = clf.decision_function(features)
keep = non_max_suppression(boxes, scores)

for i in keep:
    x1, y1, x2, y2 = boxes[i]
    label = "object" if clf.predict([features[i]])[0] == 1 else "background"
    cv2.rectangle(test_image, (x1, y1), (x2, y2), (255, 0, 0), 2)
    cv2.putText(test_image, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 255), 2)

plt.figure(figsize=(10, 8))
plt.imshow(test_image)
plt.axis("off")
plt.title("Object Detection (Trained R-CNN + NMS)")
plt.show()